In [1]:
import numpy as np
import os
from PIL import Image

from keras.preprocessing import image
from keras.layers import Input, Conv2D,  Dropout,Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam,Adadelta
from keras.losses import binary_crossentropy
import numpy.random as rng
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [2]:
path ="/home/wei/data/fifth/train"
label = "/home/wei/data/fifth/label.txt"


height = 45
width = 45
dirlist = []
for root, dirs, files in os.walk(path):
    for dir in dirs :
        dirlist.append(dir)
with open(label) as file_object:
    p = file_object.readlines()




In [8]:
num = 0
X1 = np.zeros((len(dirlist)*8,45,45,3), dtype = np.uint8)
X2 = np.zeros((len(dirlist)*8,45,45,3), dtype = np.uint8)
y = np.zeros((len(dirlist)*8, 1), dtype = np.uint8)

for dir in dirlist:
    images1 = []
    images2 = []
            
    
    image1 = Image.open(path+'/'+dir+'/'+dir+'.jpg')
    image1 = image1.resize((180,45),)
                    
    for i in range(4):
        left = 45*i
        right = 45 * (i+1)
        box = (left,0,right,45)
        data = np.asarray(image1.crop(box))
        images1.append(data)
        
    for l in p:
        if l[:4] == dir:
            break;
    
    
    for i in range(4):
        
        im = np.asarray(Image.open(path+'/'+dir+'/'+l[5+i]+'.jpg'))
        X1[num] = images1[i]
        X2[num] = im
        y[num] = 1
        num += 1
        im = np.asarray(Image.open(path+'/'+dir+'/'+str((int(l[5+i])+8)%9)+'.jpg'))                           
        X1[num] = images1[i]
        X2[num] = im
        num += 1
        
        


In [9]:
#保存成pkl文件
file = open('/home/wei/data/fifth/f_data/data8.pkl','wb')
pickle.dump(([X1,X2],y) , file)
print("ok")

ok


In [2]:
#读取pickle文件
file = open('/home/wei/data/fifth/f_data/data8.pkl', 'rb')
[X1,X2], y = pickle.load(file)
print("ok")

FileNotFoundError: [Errno 2] No such file or directory: '/home/wei/data/fifth/vval/data1.pkl'

In [3]:
input_shape = (45, 45, 3)    
left_input = Input(input_shape)
right_input = Input(input_shape)
convnet = Sequential()

convnet.add(Conv2D(32,(7,7),padding='same',activation='relu',input_shape=input_shape ))
convnet.add(Conv2D(32,(5,5),padding='same',activation='relu'))
convnet.add(MaxPooling2D())

convnet.add(Conv2D(64,(5,5),padding='same',activation='relu'))
convnet.add(Conv2D(64,(3,3),padding='same',activation='relu'))
convnet.add(MaxPooling2D())

convnet.add(Conv2D(128,(3,3),padding='same',activation='relu'))
convnet.add(Conv2D(256,(3,3),padding='same',activation='relu'))
convnet.add(MaxPooling2D())

convnet.add(Flatten())
convnet.add(Dropout(0.35))
convnet.add(Dense(2048,activation="relu"))
                  
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))        

L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid')(L1_distance)  
model = Model(input = [left_input,right_input],outputs=prediction)


optimizer = Adadelta(lr=0.1)

model.compile(loss="binary_crossentropy",optimizer=optimizer,metrics = ['accuracy'])  

checkpointer = ModelCheckpoint(filepath="/home/wei/weight_5.3/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", 
                               verbose=2, save_weights_only=True)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


In [10]:
model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)


# file = open('/home/wei/data/fifth/f_data/data5.pkl', 'rb')
# [X1,X2], y = pickle.load(file)
# model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)


# file = open('/home/wei/data/fifth/f_data/data4.pkl', 'rb')
# [X1,X2], y = pickle.load(file)
# model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)

# file = open('/home/wei/data/fifth/f_data/data3.pkl', 'rb')
# [X1,X2], y = pickle.load(file)
# model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)

# file = open('/home/wei/data/fifth/f_data/data2.pkl', 'rb')
# [X1,X2], y = pickle.load(file)
# model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)


# file = open('/home/wei/data/fifth/f_data/data1.pkl', 'rb')
# [X1,X2], y = pickle.load(file)
# model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)


# # file = open('/home/wei/data/fifth/f_data/data4.pkl', 'rb')
# # [X1,X2], y = pickle.load(file)
# # model.fit([X1,X2], y,epochs=2,callbacks=[checkpointer],validation_split=0.1)




Train on 69840 samples, validate on 7760 samples
Epoch 1/2
69824/69840 [============================>.] - ETA: 0s - loss: 0.0419 - acc: 0.9873
Epoch 00001: saving model to /home/wei/weight_5.3/weights.01--0.11-0.9589.hdf5
69840/69840 [==============================] - 3956s 57ms/step - loss: 0.0419 - acc: 0.9873 - val_loss: 0.1103 - val_acc: 0.9589
Epoch 2/2
69824/69840 [============================>.] - ETA: 0s - loss: 0.0289 - acc: 0.9906
Epoch 00002: saving model to /home/wei/weight_5.3/weights.02--0.15-0.9465.hdf5
69840/69840 [==============================] - 3904s 56ms/step - loss: 0.0289 - acc: 0.9906 - val_loss: 0.1481 - val_acc: 0.9465


In [11]:
model.save_weights('/home/wei/data/fifth/data_model_weights.h5')
model.save('/home/wei/data/fifth/data_model.h5')